In [77]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, ConvLSTM2D, Conv3D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
import data_preprocessing as dpp
from importlib import reload
reload(dpp)

tf.random.set_seed(1)
np.random.seed(1)

# Data Preprocessing

In [73]:
img_folder = 'data/data1-(8.7μm)'

img_tensors = dpp.preprocess_images(img_folder, num_images=715)

print(img_tensors)

[[[[0.89803922 0.89803922 0.89803922 1.        ]
   [0.89411765 0.89411765 0.89411765 1.        ]
   [0.89411765 0.89411765 0.89411765 1.        ]
   ...
   [0.98823529 0.98823529 0.98823529 1.        ]
   [1.         1.         0.         1.        ]
   [0.98431373 0.98431373 0.98431373 1.        ]]

  [[0.90980392 0.90980392 0.90980392 1.        ]
   [0.89411765 0.89411765 0.89411765 1.        ]
   [0.89411765 0.89411765 0.89411765 1.        ]
   ...
   [0.98823529 0.98823529 0.98823529 1.        ]
   [1.         1.         0.         1.        ]
   [0.98431373 0.98431373 0.98431373 1.        ]]

  [[0.90980392 0.90980392 0.90980392 1.        ]
   [0.91372549 0.91372549 0.91372549 1.        ]
   [0.91372549 0.91372549 0.91372549 1.        ]
   ...
   [0.99215686 0.99215686 0.99215686 1.        ]
   [1.         1.         0.         1.        ]
   [0.98823529 0.98823529 0.98823529 1.        ]]

  ...

  [[0.58823529 0.58823529 0.58823529 1.        ]
   [0.89803922 0.89803922 0.0431372

In [78]:
temp_img_list = img_tensors[:715]
print(temp_img_list.shape[:])
dpp.split_images(temp_img_list)

(715, 850, 850, 4)
1p를 생성합니다.
(15, 850, 850, 4) ---> 이야...
남은 놈!: (700, 850, 850, 4)
2p를 생성합니다.
(15, 850, 850, 4) ---> 이야...
남은 놈!: (685, 850, 850, 4)
3p를 생성합니다.
(15, 850, 850, 4) ---> 이야...
남은 놈!: (670, 850, 850, 4)
4p를 생성합니다.
(15, 850, 850, 4) ---> 이야...
남은 놈!: (655, 850, 850, 4)
5p를 생성합니다.
(15, 850, 850, 4) ---> 이야...
남은 놈!: (640, 850, 850, 4)
6p를 생성합니다.
(15, 850, 850, 4) ---> 이야...
남은 놈!: (625, 850, 850, 4)
7p를 생성합니다.
(15, 850, 850, 4) ---> 이야...
남은 놈!: (610, 850, 850, 4)
8p를 생성합니다.
(15, 850, 850, 4) ---> 이야...
남은 놈!: (595, 850, 850, 4)
9p를 생성합니다.
(15, 850, 850, 4) ---> 이야...
남은 놈!: (580, 850, 850, 4)
10p를 생성합니다.
(15, 850, 850, 4) ---> 이야...
남은 놈!: (565, 850, 850, 4)
11p를 생성합니다.
(15, 850, 850, 4) ---> 이야...
남은 놈!: (550, 850, 850, 4)
12p를 생성합니다.
(15, 850, 850, 4) ---> 이야...
남은 놈!: (535, 850, 850, 4)
13p를 생성합니다.
(15, 850, 850, 4) ---> 이야...
남은 놈!: (520, 850, 850, 4)
14p를 생성합니다.
(15, 850, 850, 4) ---> 이야...
남은 놈!: (505, 850, 850, 4)
15p를 생성합니다.
(15, 850, 850, 4) ---> 이야...
남은 놈!: (490

[array([[[[0.89803922, 0.89803922, 0.89803922, 1.        ],
          [0.89411765, 0.89411765, 0.89411765, 1.        ],
          [0.89411765, 0.89411765, 0.89411765, 1.        ],
          ...,
          [0.98823529, 0.98823529, 0.98823529, 1.        ],
          [1.        , 1.        , 0.        , 1.        ],
          [0.98431373, 0.98431373, 0.98431373, 1.        ]],
 
         [[0.90980392, 0.90980392, 0.90980392, 1.        ],
          [0.89411765, 0.89411765, 0.89411765, 1.        ],
          [0.89411765, 0.89411765, 0.89411765, 1.        ],
          ...,
          [0.98823529, 0.98823529, 0.98823529, 1.        ],
          [1.        , 1.        , 0.        , 1.        ],
          [0.98431373, 0.98431373, 0.98431373, 1.        ]],
 
         [[0.90980392, 0.90980392, 0.90980392, 1.        ],
          [0.91372549, 0.91372549, 0.91372549, 1.        ],
          [0.91372549, 0.91372549, 0.91372549, 1.        ],
          ...,
          [0.99215686, 0.99215686, 0.99215686, 1.

In [80]:
plt.figure(figsize=(30, 100))

for i in range(len(img_tensors)):
    plt.subplot(143, 5, i + 1)
    plt.imshow(img_tensors[i])
    plt.axis('off')

plt.show()

KeyboardInterrupt: 

# Modeling

In [3]:
# 자 모델 구축 들갑니다~
model = Sequential()
model.add(ConvLSTM2D(filters=40, 
                     kernel_size=(3,3), 
                     input_shape=(None,850,850,3), 
                     padding="same", 
                     return_sequences=True))
model.add(BatchNormalization())
model.add(ConvLSTM2D(filters=40, 
                     kernel_size=(3,3), 
                     padding="same", 
                     return_sequences=True))
model.add(BatchNormalization())
model.add(ConvLSTM2D(filters=40, 
                     kernel_size=(3,3), 
                     padding="same", 
                     return_sequences=True))
model.add(BatchNormalization())
model.add(ConvLSTM2D(filters=40, 
                     kernel_size=(3,3), 
                     padding="same", 
                     return_sequences=True))
model.add(BatchNormalization())
model.add(ConvLSTM2D(filters=40, 
                     kernel_size=(3,3), 
                     padding="same", 
                     return_sequences=True))
model.add(BatchNormalization())
model.add(Conv3D(filters=1, 
                 kernel_size=(3,3,3), 
                 activation="sigmoid",
                 padding="same"))
model.compile(loss='binary_crossentropy',optimizer='adadelta', metrics = ['accuracy'])
callbacks_val_acc = [EarlyStopping(monitor='val_accuracy', patience=3, mode='max', restore_best_weights=True),
                     ModelCheckpoint("ClimateAI.keras", monitor='val_accuracy', save_best_only=True)]
callbacks_val_loss = [EarlyStopping(monitor='val_loss', patience=3, mode='min', restore_best_weights=True),
                      ModelCheckpoint("ClimateAI.keras", monitor='val_loss', save_best_only=True)]
callbacks = [callbacks_val_acc, callbacks_val_loss]

model.summary()

/Volumes/looparam/#1 Coding/#10 Python/AcademicFestival/AF/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_lstm2d (ConvLSTM2D)        │ (None, None, 850, 850, │        62,080 │
│                                 │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, None, 850, 850, │           160 │
│ (BatchNormalization)            │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_1 (ConvLSTM2D)      │ (None, None, 850, 850, │       115,360 │
│                                 │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, None, 850, 850, │           160 │
│ (BatchNormalization)            │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_2 (ConvLSTM2D)      │ (None, None, 850, 850, │       115,360 │
│                                 │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, None, 850, 850, │           160 │
│ (BatchNormalization)            │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_3 (ConvLSTM2D)      │ (None, None, 850, 850, │       115,360 │
│                                 │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, None, 850, 850, │           160 │
│ (BatchNormalization)            │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_4 (ConvLSTM2D)      │ (None, None, 850, 850, │       115,360 │
│                                 │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, None, 850, 850, │           160 │
│ (BatchNormalization)            │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d (Conv3D)                 │ (None, None, 850, 850, │         1,081 │
│                                 │ 1)                     │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 525,401 (2.00 MB)

 Trainable params: 525,001 (2.00 MB)

 Non-trainable params: 400 (1.56 KB)

# Train

In [ ]:
model.fit(img_tensors, epochs=10, batch_size=5, validation_split=0.1)